# Project 2

You have been approached by The Director of Yemisi Shyllon Museum of Art (YSMA) to build a computer vision program with a graphical user interface (GUI) that would ask for a username and password from visitors to log in and view and perform image enhancements on the art collection at the museum. Given the knowledge of the collection categories from the previous project.

Hint: Use the Python Tinker library to build your GUI


In [1]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from PIL import ImageTk, Image

### Enhancement Functions

In [2]:
# Function for enhancing image using blue channel
def enhance_blue_channel(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    B, _, _ = cv2.split(image)
    return B

# Function for enhancing image using green channel
def enhance_green_channel(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    _, G, _ = cv2.split(image)
    return G

# Function for enhancing image using red channel
def enhance_red_channel(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    _, _, R = cv2.split(image)
    return R

# Function for enhancing image using weighted addition
def enhance_weighted_addition(image_path1, image_path2):
    image1 = cv2.imread(image_path1)
    image2 = cv2.imread(image_path2)
    if image1 is None or image2 is None:
        return None
    # Resize images
    image1 = cv2.resize(image1, (500,400))
    image2 = cv2.resize(image2, (500,400))
    return cv2.addWeighted(image1, 0.5, image2, 0.6, 0)

# Function for enhancing image using subtraction
def enhance_subtraction(image_path1, image_path2):
    image1 = cv2.imread(image_path1)
    image2 = cv2.imread(image_path2)
    if image1 is None or image2 is None:
        return None
    # Resize images
    image1 = cv2.resize(image1, (500,400))
    image2 = cv2.resize(image2, (500,400))
    return cv2.subtract(image1, image2)

# Function for enhancing image using sharpening
def enhance_sharpening(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# Function for enhancing image using median blur
def enhance_median_blur(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    return cv2.medianBlur(image, 15)

# Function for enhancing image by scaling
def enhance_scaling(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    return cv2.resize(image, None, fx=2, fy=2)

# Function for enhancing image by adjusting brightness and contrast
def enhance_brightness_contrast(image_path, brightness=5, contrast=1.5):
    image = cv2.imread(image_path)
    if image is None:
        return None
    return cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)

# Function for enhancing image by inverting colors
def enhance_inverse(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    return 255 - image

### Transformation Functions

In [3]:
def translate_image(image_path, dx, dy):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    translated_img = cv2.warpAffine(img, M, (cols, rows))
    
    return img, translated_img
    
def reflect_image(image_path):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, 0, 0], [0, -1, rows]])
    reflected_img = cv2.warpAffine(img, M, (cols, rows))

    return img, reflected_img

def rotate_image(image_path, angle, scale):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, scale)
    rotated_img = cv2.warpAffine(img, M, (cols, rows))

    return img, rotated_img

def crop_image(image_path, x1, y1, x2, y2):
    img = cv2.imread(image_path, 0)
    cropped_img = img[y1:y2, x1:x2]
    
    return img, cropped_img

def shear_image_x(image_path, shear_factor):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, shear_factor, 0], [0, 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M, (int(cols*1.5), int(rows*1.5)))
    
    return img, sheared_img

def shear_image_y(image_path, shear_factor):
    img = cv2.imread(image_path, 0)
    rows, cols = img.shape
    
    M = np.float32([[1, 0, 0], [shear_factor, 1, 0], [0, 0, 1]])
    sheared_img = cv2.warpPerspective(img, M, (int(cols*1.5), int(rows*1.5)))
    
    return img, sheared_img

def apply_blur(image_path, blur_type):
    img = cv2.imread(image_path)
    
    if blur_type == 'Gaussian Blur':
        blurred_img = cv2.GaussianBlur(img, (7, 7), 0)
    elif blur_type == 'Median Blur':
        blurred_img = cv2.medianBlur(img, 5)
    elif blur_type == 'Bilateral Blur':
        blurred_img = cv2.bilateralFilter(img, 9, 75, 75)
    else:
        print("Invalid blur type.")
        return
    return img, blurred_img



In [4]:
def login(root, username_entry, password_entry):
    username = username_entry.get()
    password = password_entry.get()

    if username == "admin" and password == "password":
        messagebox.showinfo("Login Successful", "Welcome, Admin!")
        root.destroy()  
        open_enhancement_transformation_window()  
    else:
        messagebox.showerror("Login Failed", "Invalid username or password.")


In [5]:
def open_image(original_image_label):
    global selected_image_path
    selected_image_path = None
    
    filename = filedialog.askopenfilename()
    if filename:
        selected_image_path = filename
        
        img = cv2.imread(filename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = Image.fromarray(img)
        img.thumbnail((300, 400))
        img = ImageTk.PhotoImage(img)
        
        original_image_label.config(image=img)
        original_image_label.image = img

        
# Center a window on the screen.
def center_window(window):
    window.update_idletasks()
    
    width = window.winfo_width()
    height = window.winfo_height()
    
    x_offset = (window.winfo_screenwidth() - width) // 2
    y_offset = (window.winfo_screenheight() - height) // 2
    
    window.geometry(f"{width}x{height}+{x_offset}+{y_offset}")


### Display Enhanced Image Function

In [6]:
def display_enhanced_image(enhanced_img, enhanced_image_label):
    # Check if the image is a color image
    is_color_image = len(enhanced_img.shape) == 3 and enhanced_img.shape[2] == 3

    # Convert BGR image to RGB if it's a color image
    if is_color_image:
        enhanced_img_rgb = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB)
    else:
        enhanced_img_rgb = enhanced_img

    # Convert NumPy array to PIL Image
    enhanced_img_pil = Image.fromarray(enhanced_img_rgb)

    if not is_color_image:
        enhanced_img_pil = enhanced_img_pil.convert("L")

    # Resize the image to fit the label
    enhanced_img_pil.thumbnail((300, 400))

    # Convert PIL Image to PhotoImage
    enhanced_img_tk = ImageTk.PhotoImage(enhanced_img_pil)

    # Display the enhanced image in the label
    enhanced_image_label.config(text="Enhanced Image")
    enhanced_image_label.config(image=enhanced_img_tk)
    enhanced_image_label.image = enhanced_img_tk

    # Pack the label to update the display
    enhanced_image_label.pack(expand=True, fill="both")



### Display Transformed Image Function

In [7]:
#  Display the transformed image in the transformation window.
def display_transformed_image(transformed_img, transformation_parameters_window, transformed_image_label):

    # Check if the image is grayscale
    is_grayscale = len(transformed_img.shape) == 2 or transformed_img.shape[2] == 1

    # Convert BGR image to RGB for display if not grayscale
    if not is_grayscale:
        transformed_img_rgb = cv2.cvtColor(transformed_img, cv2.COLOR_BGR2RGB)
    else:
        transformed_img_rgb = transformed_img

    # Convert NumPy array to PIL Image
    transformed_img_pil = Image.fromarray(transformed_img_rgb)

    # Resize the image to fit the label
    transformed_img_pil.thumbnail((300, 400))  

    # Convert PIL Image to PhotoImage
    transformed_img_tk = ImageTk.PhotoImage(transformed_img_pil)

    # Display the transformed image in the right-side frame
    transformed_image_label.config(text="Transformed Image")
    transformed_image_label.config(image=transformed_img_tk)
    transformed_image_label.image = transformed_img_tk

    # Pack the label to update the display
    transformed_image_label.pack(expand=True, fill="both")

    # Destroy the transformation window
    transformation_parameters_window.destroy()

### Open Transformation Parameters Window Function

In [8]:
# Define entry fields and other widgets globally
transformation_entries = {}

def open_transformation_parameters_window(transformation_window, transformed_image_label, transformation_dropdown):
    global transformation_entries
    
     # Clear the transformation_entries dictionary
    transformation_entries.clear()
    
    if selected_image_path is not None:

        # Create a new window for transformation parameters
        transformation_parameters_window = tk.Toplevel(transformation_window)
        transformation_parameters_window.title("YSMA Image Enhancement & Transformation App | Transformation Parameters")
        transformation_parameters_window.geometry("600x600")
        transformation_parameters_window.resizable(False, False)

        # Center the transformation parameters window on the screen
        center_window(transformation_parameters_window)

        selected_transformation = transformation_dropdown.get()

        # Define parameter labels and entry fields for each transformation
        parameters = {
            "Translate": ["dx", "dy"],
            "Reflect": [],
            "Rotate": ["Angle", "Scale"],
            "Shear X": ["Shear Factor"],
            "Shear Y": ["Shear Factor"],
            "Crop": ["x1", "y1", "x2", "y2"]
        }

        # Add widgets based on the selected transformation
        if selected_transformation in parameters:
            ttk.Label(transformation_parameters_window, text="Transformation Parameters").pack(pady=10)

            for param in parameters[selected_transformation]:
                ttk.Label(transformation_parameters_window, text=param + ":").pack(pady=5)
                entry = ttk.Entry(transformation_parameters_window)
                entry.pack(pady=5)
                transformation_entries[param] = entry

            # Button to apply transformation
            ttk.Button(transformation_parameters_window, text="Apply Transformation", command=lambda: perform_transformation(selected_transformation, transformation_parameters_window, transformed_image_label)).pack(pady=10)

        elif selected_transformation in ["Gaussian Blur", "Median Blur", "Bilateral Blur"]:
            ttk.Button(transformation_parameters_window, text="Apply Blur", command=lambda: perform_transformation(selected_transformation, transformation_parameters_window, transformed_image_label)).pack(pady=10)

        else:
            ttk.Label(transformation_parameters_window, text="Transformation not yet implemented").pack(pady=10)
    
    else:
        messagebox.showinfo("Info", "No image selected.")


### Perform Enchancement Function

In [9]:
def perform_enhancement(selected_enhancement, enhanced_image_label):
    # Load the "superman.jpg" image
    superman_image_path = "../images/project-images/superman.jpg"

    # Apply the selected enhancement based on its name
    if selected_enhancement == "Blue Channel":
        enhanced_image = enhance_blue_channel(selected_image_path)
    elif selected_enhancement == "Green Channel":
        enhanced_image = enhance_green_channel(selected_image_path)
    elif selected_enhancement == "Red Channel":
        enhanced_image = enhance_red_channel(selected_image_path)
    elif selected_enhancement == "Weighted Addition":
        enhanced_image = enhance_weighted_addition(selected_image_path, superman_image_path)
    elif selected_enhancement == "Subtraction":
        enhanced_image = enhance_subtraction(selected_image_path, superman_image_path)
    elif selected_enhancement == "Sharpening":
        enhanced_image = enhance_sharpening(selected_image_path)
    elif selected_enhancement == "Median Blur":
        enhanced_image = enhance_median_blur(selected_image_path)
    elif selected_enhancement == "Scaling":
        enhanced_image = enhance_scaling(selected_image_path)
    elif selected_enhancement == "Brightness & Contrast Adjustment":
        enhanced_image = enhance_brightness_contrast(selected_image_path)
    elif selected_enhancement == "Inverse Color":
        enhanced_image = enhance_inverse(selected_image_path)
    else:
        messagebox.showerror("Error", "Invalid enhancement selected.")
        return

    # Display the enhanced image
    if enhanced_image is not None:
        display_enhanced_image(enhanced_image, enhanced_image_label)


### Perform Tranformation Function

In [10]:
def perform_transformation(selected_transformation, transformation_parameters_window, transformed_image_label):
    
    # Check if the selected transformation requires parameters
    transformations_with_params = ["Translate", "Rotate", "Shear X", "Shear Y", "Crop"]

    if selected_transformation in transformations_with_params and not transformation_entries:
        messagebox.showerror("Error", "No transformation parameters entered.")
        return


    # Get the selected transformation and its parameters
    parameters = {param: entry.get() for param, entry in transformation_entries.items()}

    # Apply the selected transformation based on its name
    if selected_transformation == "Translate":
        img, transformed_img = translate_image(selected_image_path, int(parameters["dx"]), int(parameters["dy"]))
    elif selected_transformation == "Reflect":
        img, transformed_img = reflect_image(selected_image_path)
    elif selected_transformation == "Rotate":
        img, transformed_img = rotate_image(selected_image_path, float(parameters["Angle"]), float(parameters["Scale"]))
    elif selected_transformation == "Shear X":
        img, transformed_img = shear_image_x(selected_image_path, float(parameters["Shear Factor"]))
    elif selected_transformation == "Shear Y":
        img, transformed_img = shear_image_y(selected_image_path, float(parameters["Shear Factor"]))
    elif selected_transformation == "Crop":
        img, transformed_img = crop_image(selected_image_path, int(parameters["x1"]), int(parameters["y1"]), int(parameters["x2"]), int(parameters["y2"]))
    elif selected_transformation in ["Gaussian Blur", "Median Blur", "Bilateral Blur"]:
        img, transformed_img = apply_blur(selected_image_path, selected_transformation)
    else:
        messagebox.showerror("Error", "Invalid transformation selected.")
        return

    # Display the transformed image
    if transformed_img is not None:
        display_transformed_image(transformed_img, transformation_parameters_window, transformed_image_label)

### Function To Open Window For Imgage Enhancement & Tranformation

In [11]:
def open_enhancement_transformation_window():
    # Create the enhancement_transformation_window
    enhancement_transformation_window = tk.Tk()
    enhancement_transformation_window.title("YSMA Image Enhancement & Transformation App")

    # Get the screen width and height
    screen_width = enhancement_transformation_window.winfo_screenwidth()
    screen_height = enhancement_transformation_window.winfo_screenheight()

    # Define margin size
    margin = 50

    # Set the geometry with margin
    enhancement_transformation_window.geometry(f"{screen_width - 2 * margin}x{screen_height - 2 * margin}+{margin}+{margin}")

    enhancement_transformation_window.resizable(False, False)

    # Center the window on the screen
    center_window(enhancement_transformation_window)
    
    style = ttk.Style()
    style.theme_use("vista")
    style.configure("TLabel", padding=6)
    style.configure("TEntry", padding=6)
    style.configure("TButton", padding=6)

    # Create a frame for the left section (original image)
    left_frame = ttk.Frame(enhancement_transformation_window, width=enhancement_transformation_window.winfo_width() // 3, borderwidth=2, relief="solid")
    left_frame.pack(side="left", expand=True, fill="both")

    # Label for original image title
    original_image_title_label = ttk.Label(left_frame, text="Original Image", font=("Helvetica", 16, "bold"))
    original_image_title_label.pack(side="top", padx=10, pady=(10, 5))

    # Label for original image
    original_image_label = ttk.Label(left_frame, borderwidth=2, relief="solid")
    original_image_label.pack(expand=True, fill="both")

    # Button to select original image
    select_image_button = ttk.Button(left_frame, text="Select Original Image", command=lambda: open_image(original_image_label))
    select_image_button.pack(pady=30)

    # Create a frame for the middle section (enhanced image)
    middle_frame = ttk.Frame(enhancement_transformation_window, width=enhancement_transformation_window.winfo_width() // 3, borderwidth=2, relief="solid")
    middle_frame.pack(side="left", expand=True, fill="both")

    # Label for enhanced image title
    enhanced_image_title_label = ttk.Label(middle_frame, text="Enhanced Image", font=("Helvetica", 16, "bold"))
    enhanced_image_title_label.pack(side="top", padx=10, pady=(10, 5))

    # Label for enhanced image
    enhanced_image_label = ttk.Label(middle_frame, borderwidth=2, relief="solid")
    enhanced_image_label.pack(expand=True, fill="both")

    # Label for selecting enhancement
    enhancement_label = ttk.Label(middle_frame, text="Select Enhancement:")
    enhancement_label.pack(side="left", padx=(20, 10), pady=30, anchor="w")

    enhancement_choices = ["Blue Channel", "Green Channel", "Red Channel", "Weighted Addition", "Subtraction", "Sharpening", "Median Blur", "Scaling", "Brightness & Contrast Adjustment", "Inverse Color"]
    enhancement_dropdown = ttk.Combobox(middle_frame, values=enhancement_choices, state="readonly")
    enhancement_dropdown.pack(side="left", padx=(0, 10), pady=5)

    # Button to open enhancement/transformation parameters window
    open_enhancement_transformation_button = ttk.Button(middle_frame, text="Enhance", command=lambda: perform_enhancement(enhancement_dropdown.get(), enhanced_image_label))
    open_enhancement_transformation_button.pack(side="right", padx=(10, 20), pady=30)

    # Create a frame for the right section (transformed image)
    right_frame = ttk.Frame(enhancement_transformation_window, width=enhancement_transformation_window.winfo_width() // 3, borderwidth=2, relief="solid")
    right_frame.pack(side="left", expand=True, fill="both")

    # Label for transformed image title
    transformed_image_title_label = ttk.Label(right_frame, text="Transformed Image", font=("Helvetica", 16, "bold"))
    transformed_image_title_label.pack(side="top", padx=10, pady=(10, 5))

    # Label for transformed image
    transformed_image_label = ttk.Label(right_frame, borderwidth=2, relief="solid")
    transformed_image_label.pack(expand=True, fill="both")

    # Label for selecting transformation
    transformation_label = ttk.Label(right_frame, text="Select Transformation:")
    transformation_label.pack(side="left", padx=(20, 10), pady=30, anchor="w")

    transformation_choices = ["Translate", "Reflect", "Rotate", "Crop", "Shear X", "Shear Y", "Gaussian Blur", "Median Blur", "Bilateral Blur"]
    transformation_dropdown = ttk.Combobox(right_frame, values=transformation_choices, state="readonly")
    transformation_dropdown.pack(side="left", padx=(0, 10), pady=5)

    # Button to open transformation window
    open_transformation_button = ttk.Button(right_frame, text="Open Transformation Window", command=lambda: open_transformation_parameters_window(enhancement_transformation_window, transformed_image_label, transformation_dropdown))
    open_transformation_button.pack(side="right", padx=(10, 20), pady=30)


### Main Application Entry

In [12]:
def ysma_image_enhancement_transformation_app():
    root = tk.Tk()
    root.title("YSMA Image Enhancement & Transformation App | Login")
    root.geometry("400x200")

    center_window(root)
    root.resizable(False, False)

    # Apply a modern style
    style = ttk.Style()
    style.theme_use("vista")
    style.configure("TLabel", padding=6)
    style.configure("TEntry", padding=6)
    style.configure("TButton", padding=6)

    # Login Frame
    login_frame = ttk.Frame(root)
    login_frame.pack(expand=True, padx=20, pady=20)

    # Username Label and Entry
    username_label = ttk.Label(login_frame, text="Username:")
    username_label.grid(row=0, column=0, sticky=tk.W)

    username_entry = ttk.Entry(login_frame, width=30)
    username_entry.grid(row=0, column=1, sticky=tk.W, pady=5)

    # Password Label and Entry
    password_label = ttk.Label(login_frame, text="Password:")
    password_label.grid(row=1, column=0, sticky=tk.W)

    password_entry = ttk.Entry(login_frame, show="*", width=30)
    password_entry.grid(row=1, column=1, sticky=tk.W, pady=5)

    # Login Button
    login_button = ttk.Button(login_frame, text="Login", command= lambda:login(root, username_entry, password_entry), width=30)
    login_button.grid(row=2, column=0, columnspan=2, pady=(20, 0))

    # Start the Tkinter event loop
    root.mainloop()

In [13]:
ysma_image_enhancement_transformation_app()